In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {'hello' : 0, 'world' : 1}
embedds = nn.Embedding(2, 5)
lookup_tensor = torch.tensor([word_to_ix['hello']], dtype = torch.long)
hello_embedd = embedds(lookup_tensor)
print("Printing Word Embedding for \"hello\" : {}".format(hello_embedd))

Printing Word Embedding for "hello" : tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]])
